In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

In [2]:
ds = pd.read_csv("train.csv")
df = ds.sample(frac=1)

In [3]:
df.shape

(74, 2)

In [4]:
df.head()

,valor,texto
6,0,O presente estudo tem como objetivo analisar o...
39,1,É constante e mundialmente difundida a busca p...
51,1,Devido aos impactos causados pela geração de e...
36,1,Pode-se dizer que na grande área da engenharia...
47,1,Este trabalho apresenta um estudo do acionamen...


In [5]:
print((df.valor==1).sum())#eletrica
print((df.valor==0).sum())#direito

37
37


In [6]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",text)

def remove_punct(text):
    translator = str.maketrans("","",'!""#$%&\'()*+,./:;<=>?@[\\]^_`{|}~º')
    return text.translate(translator)

def remove_hifen(text):
    translator = str.maketrans('-',' ')
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
pattern = re.compile(r"https?//(\S+|www)\.\S+")
for t in df.texto:
    matches = pattern.findall(t)
    for match in  matches:
        print(t)
        print(match)
        print(pattern.sub(r"",t))
        
    if len(matches)> 0:
        break

In [8]:
df["texto"] = df.texto.map(remove_URL)
df["texto"] = df.texto.map(remove_punct)
df["texto"] = df.texto.map(remove_hifen)

In [9]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("portuguese"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mateus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


In [11]:
df["texto"] = df.texto.map(remove_stopwords)

In [12]:
df.texto

6     presente estudo objetivo analisar incidente re...
39    constante mundialmente difundida busca sistema...
51    devido impactos causados geração energia elétr...
36    pode dizer grande área engenharia elétrica qua...
47    trabalho apresenta estudo acionamento motores ...
                            ...                        
38    sucesso empresa associado capacidade entender ...
26    presente trabalho escopo analisar autocomposiç...
43    conversores cc ca elementos eletrônica potênci...
15    presente trabalho dedica estudo importância po...
34    presente trabalho apresenta desenvolvimento po...
Name: texto, Length: 74, dtype: object

In [13]:
df.texto[72]

'tomando cenário mundo globalizado assistimos efervescência comércio internacional especificamente brasil buscado projetar potência econômica pesquisa pretende contribuir discutir problematizar nesse contexto global inseridos necessidade medidas regulamentem práticas comércio exterior assim presente pesquisa tema estudo sobre inovações decreto n 8058 26 julho 2013 procedimentos administrativos relativos investigação aplicação medidas antidumping pretende tomar foco modificações relação decreto anterior decreto n 160295 pautando três pilares celeridade investigações eficácia medidas aplicadas previsibilidade setor privado implicando aspectos protecionistas tanto abordadas generalidades defesa comercial fundamentos requisitos órgãos envolvidos imposição medidas defesa comercial tendo levantado procedimentos administrativos feita análise alertando sobre consequências novo decreto protecionista ver pode trazer cenário econômico nacional consumidores sempre sofrem forma final direta medidas

In [14]:
from collections import Counter

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

counter = counter_word(df.texto)

In [15]:
len(counter)

3224

In [16]:
counter.most_common(5)

[('trabalho', 86),
 ('motores', 57),
 ('estudo', 48),
 ('direito', 46),
 ('sistema', 38)]

In [17]:
num_unique_words = len(counter)

In [18]:
train_size = 56

train_df = df[:train_size]
val_df = df[train_size:]

In [19]:
print(len(train_df))
print(len(val_df))

56
18


In [20]:
train_sentences = train_df.texto.to_numpy()
train_labels = train_df.valor.to_numpy()

val_sentences = val_df.texto.to_numpy()
val_labels = val_df.valor.to_numpy()

In [21]:
train_sentences.shape, val_sentences.shape

((56,), (18,))

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = num_unique_words,oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

In [23]:
word_index = tokenizer.word_index

In [24]:
word_index

{'<OOV>': 1,
 'trabalho': 2,
 'motores': 3,
 'estudo': 4,
 'motor': 5,
 'sistema': 6,
 'indução': 7,
 'forma': 8,
 'análise': 9,
 'ser': 10,
 'controle': 11,
 'desenvolvimento': 12,
 'direito': 13,
 'objetivo': 14,
 'corrente': 15,
 'fim': 16,
 'sobre': 17,
 'robô': 18,
 'presente': 19,
 'rotor': 20,
 'partir': 21,
 'proteção': 22,
 'energia': 23,
 'parte': 24,
 'geração': 25,
 'tanto': 26,
 'ambiente': 27,
 'projeto': 28,
 'aplicação': 29,
 'regime': 30,
 'modelo': 31,
 'bem': 32,
 'ambiental': 33,
 'analisar': 34,
 'elétrica': 35,
 'assim': 36,
 'cada': 37,
 'resultados': 38,
 'através': 39,
 'testes': 40,
 'pode': 41,
 'velocidade': 42,
 'dois': 43,
 'melhor': 44,
 'utilizando': 45,
 'mercado': 46,
 'tensão': 47,
 'meio': 48,
 'relação': 49,
 'ainda': 50,
 'custo': 51,
 'sistemas': 52,
 'simulações': 53,
 'países': 54,
 'brasil': 55,
 'custos': 56,
 'características': 57,
 'software': 58,
 'tema': 59,
 'lei': 60,
 'processo': 61,
 'brasileiro': 62,
 'sendo': 63,
 'busca': 64,
 'pode

In [25]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [26]:
#print(train_sentences[10:15])
#print(train_sequences[10:15])

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 100

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding = "post",truncating = "post")
val_padded = pad_sequences(val_sequences, maxlen = max_length, padding = "post", truncating = "post")
train_padded.shape, val_padded.shape

((56, 100), (18, 100))

In [28]:
train_padded[10]

array([   2,   14, 1244,  690,   17,  691,  444,  294,  691, 1245,  295,
       1246,   12,   75,  139, 1247, 1248,  210,  296,  445,  280,  162,
        692,  297, 1249,  446,   73,   17,    3,  272,   15,  114, 1250,
        211,    4, 1251,  298, 1252,  212,  121,    4, 1253,  108,   14,
       1254,  109,  690,  211,   57,   93,  176,  163,  158,  444,    5,
        272,   15,  114, 1255,   67,   81,  158,  263,  390,  447,  116,
        192,  401, 1256, 1257,   69,   29,  210,  296,  448,  449,  693,
        188,  405,  671,   72,    3,  450,  277,  626,  286,  445,  280,
        162,  297,  694,  121,  113, 1258, 1259, 1260, 1261,  141, 1262,
       1263])

In [29]:
#print(train_sentences[10])
#print(train_sequences[10])
#print(train_padded[10])

In [30]:
reverse_word_index = dict([(idx,word) for (word, idx) in word_index.items()])

In [31]:
reverse_word_index

{1: '<OOV>',
 2: 'trabalho',
 3: 'motores',
 4: 'estudo',
 5: 'motor',
 6: 'sistema',
 7: 'indução',
 8: 'forma',
 9: 'análise',
 10: 'ser',
 11: 'controle',
 12: 'desenvolvimento',
 13: 'direito',
 14: 'objetivo',
 15: 'corrente',
 16: 'fim',
 17: 'sobre',
 18: 'robô',
 19: 'presente',
 20: 'rotor',
 21: 'partir',
 22: 'proteção',
 23: 'energia',
 24: 'parte',
 25: 'geração',
 26: 'tanto',
 27: 'ambiente',
 28: 'projeto',
 29: 'aplicação',
 30: 'regime',
 31: 'modelo',
 32: 'bem',
 33: 'ambiental',
 34: 'analisar',
 35: 'elétrica',
 36: 'assim',
 37: 'cada',
 38: 'resultados',
 39: 'através',
 40: 'testes',
 41: 'pode',
 42: 'velocidade',
 43: 'dois',
 44: 'melhor',
 45: 'utilizando',
 46: 'mercado',
 47: 'tensão',
 48: 'meio',
 49: 'relação',
 50: 'ainda',
 51: 'custo',
 52: 'sistemas',
 53: 'simulações',
 54: 'países',
 55: 'brasil',
 56: 'custos',
 57: 'características',
 58: 'software',
 59: 'tema',
 60: 'lei',
 61: 'processo',
 62: 'brasileiro',
 63: 'sendo',
 64: 'busca',
 65: '

In [32]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [33]:
decoded_text = decode(train_sequences[10])

#print(train_sequences[10])
#print(decoded_text)

In [63]:
from tensorflow.keras import layers

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length = max_length))

model.add(layers.LSTM(256,dropout = 0.1))
#model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(24, activation = "relu"))
model.add(layers.Dense(24, activation = "relu"))
#model.add(layers.Dense(24, activation = "softmax"))
#model.add(layers.Softmax())
model.add(layers.Dense(1, activation = "sigmoid"))


model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 32)           103168    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_11 (Dense)             (None, 24)                6168      
_________________________________________________________________
dense_12 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 25        
Total params: 405,897
Trainable params: 405,897
Non-trainable params: 0
_________________________________________________________________


In [64]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer = optim, metrics = metrics)

In [65]:
import time
start = time.perf_counter()
model.fit(train_padded,train_labels, epochs = 20, validation_data=(val_padded,val_labels), verbose=2)
finish = time.perf_counter()
print(f'\nFinished in {round(finish-start, 2)} second(s)')

Epoch 1/20
2/2 - 2s - loss: 0.6932 - accuracy: 0.5714 - val_loss: 0.6961 - val_accuracy: 0.3889
Epoch 2/20
2/2 - 0s - loss: 0.6921 - accuracy: 0.5000 - val_loss: 0.6990 - val_accuracy: 0.3333
Epoch 3/20
2/2 - 0s - loss: 0.6893 - accuracy: 0.5536 - val_loss: 0.7028 - val_accuracy: 0.3333
Epoch 4/20
2/2 - 0s - loss: 0.6862 - accuracy: 0.5536 - val_loss: 0.7073 - val_accuracy: 0.3333
Epoch 5/20
2/2 - 0s - loss: 0.6791 - accuracy: 0.5714 - val_loss: 0.7162 - val_accuracy: 0.3333
Epoch 6/20
2/2 - 0s - loss: 0.6664 - accuracy: 0.5714 - val_loss: 0.7191 - val_accuracy: 0.3333
Epoch 7/20
2/2 - 0s - loss: 0.6177 - accuracy: 0.6964 - val_loss: 2.1027 - val_accuracy: 0.3333
Epoch 8/20
2/2 - 0s - loss: 0.9418 - accuracy: 0.6250 - val_loss: 0.6760 - val_accuracy: 0.6667
Epoch 9/20
2/2 - 0s - loss: 0.6619 - accuracy: 0.7321 - val_loss: 0.6758 - val_accuracy: 0.6667
Epoch 10/20
2/2 - 0s - loss: 0.6711 - accuracy: 0.6786 - val_loss: 0.6743 - val_accuracy: 0.6667
Epoch 11/20
2/2 - 0s - loss: 0.6662 - a

In [66]:
predictions = model.predict(val_padded)
print(predictions)
predictions = [1 if p > 0.5 else 0 for p in predictions]

[[0.01952085]
 [0.01772228]
 [0.02178544]
 [0.4323744 ]
 [0.01820695]
 [0.58236074]
 [0.01739466]
 [0.01782724]
 [0.02173245]
 [0.01784486]
 [0.04914144]
 [0.64912057]
 [0.01792389]
 [0.0792416 ]
 [0.02002713]
 [0.5813783 ]
 [0.01700255]
 [0.5227887 ]]


In [67]:
index = 7
print(val_sentences[index])
print(val_padded[index])
print("Label: ",val_labels[index])
print("Resultado: ",predictions[index])
print(val_labels)
print(predictions)

objetivo central pesquisa analisar efetividade comissão análise solicitações especiais contexto efetivação direito saúde poder judiciário partir demandas ajuizadas defensoria pública estado paulo – regional ribeirão preto estudo buscou trazer subsídios órgão defensoria pública ampliação acesso justiça vulneráveis forma assegurar acesso bens serviços saúde indispensáveis manutenção vida digna comissão analisada formada integrantes poderes executivos municipal estadual hospital clínicas faculdade medicina ribeirão preto emite pareceres técnicos processos judiciais envolvendo pedidos medicamentos equipamentos tratamentos médicos forma racionalizar processo judicialização direito saúde evitando fornecimento medicamentos tratamentos médicos devida comprovação eficiência tratamento doença metodologia adotada pesquisa consistiu análise quantitativa dados obtidos pareceres comissão decisões liminares sentenças judiciais processos ajuizados defensoria pública envolvendo tutela direito saúde nec

In [68]:
df_t = pd.read_csv("eval.csv")

In [69]:
df_t.head()

,valor,texto
0,0,Ante ao implemento em tecnologia dos ilícitos ...
1,0,"O Objetivo é demonstrar, com a apresentação do..."
2,0,O presente estudo analisa a responsabilidade c...
3,0,É fato que a legislação brasileira adotou expr...
4,1,Este trabalho tem por objetivo analisar um mot...


In [70]:
def make_test():
    df_t["texto"] = df_t.texto.map(remove_URL)
    df_t["texto"] = df_t.texto.map(remove_punct)
    df_t["texto"] = df_t.texto.map(remove_hifen)
    df_t["texto"] = df_t.texto.map(remove_stopwords)

In [71]:
def pat():
    for t in df_t.texto:
        matches = pattern.findall(t)
        for match in  matches:
            print(t)
            print(match)
            print(pattern.sub(r"",t))        
        if len(matches)> 0:
            break

In [72]:
pat()
make_test()

In [73]:
df_t.texto

0    ante implemento tecnologia ilícitos contra ord...
1    objetivo demonstrar apresentação números ofici...
2    presente estudo analisa responsabilidade civil...
3    fato legislação brasileira adotou expressament...
4    trabalho objetivo analisar motor partida conve...
5    linguagem xml extensible markup language consi...
6    presente trabalho idealizado partir experiênci...
7    conversores cc ca elementos eletrônica potênci...
Name: texto, dtype: object

In [74]:
test_sentences = df_t.texto.to_numpy()
test_labels = df_t.valor.to_numpy()

In [75]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
word_index_test = tokenizer.word_index
word_index_test

{'<OOV>': 1,
 'trabalho': 2,
 'motores': 3,
 'estudo': 4,
 'motor': 5,
 'sistema': 6,
 'indução': 7,
 'forma': 8,
 'análise': 9,
 'ser': 10,
 'controle': 11,
 'desenvolvimento': 12,
 'direito': 13,
 'objetivo': 14,
 'corrente': 15,
 'fim': 16,
 'sobre': 17,
 'robô': 18,
 'presente': 19,
 'rotor': 20,
 'partir': 21,
 'proteção': 22,
 'energia': 23,
 'parte': 24,
 'geração': 25,
 'tanto': 26,
 'ambiente': 27,
 'projeto': 28,
 'aplicação': 29,
 'regime': 30,
 'modelo': 31,
 'bem': 32,
 'ambiental': 33,
 'analisar': 34,
 'elétrica': 35,
 'assim': 36,
 'cada': 37,
 'resultados': 38,
 'através': 39,
 'testes': 40,
 'pode': 41,
 'velocidade': 42,
 'dois': 43,
 'melhor': 44,
 'utilizando': 45,
 'mercado': 46,
 'tensão': 47,
 'meio': 48,
 'relação': 49,
 'ainda': 50,
 'custo': 51,
 'sistemas': 52,
 'simulações': 53,
 'países': 54,
 'brasil': 55,
 'custos': 56,
 'características': 57,
 'software': 58,
 'tema': 59,
 'lei': 60,
 'processo': 61,
 'brasileiro': 62,
 'sendo': 63,
 'busca': 64,
 'pode

In [76]:
print(test_sentences)
print(test_sequences)

['ante implemento tecnologia ilícitos contra ordem econômica manutenção jogo limpo players disputa mercado estado viuse obrigado adotar novos meios investigação apurar infrações momento consensualização direito busca instituto delação premiada meio prova criminal instrumento mantém ordem econômica presente trabalho objetivo demonstrar direito econômico transformou instituto delação premiada instrumento manutenção ordem econômica enfoca utilização acordos leniência instrumento combate atos anticompetitivos examina efeitos utilização acordos esfera administrativa criminal cível apurar eficiência instrumentos natureza delatória regulação mercado contrapondo objetivos atingidos principais críticas formuladas respeito prática'
 'objetivo demonstrar apresentação números oficiais divulgados infopen adoção parte governantes política criminal sanitarista injusta encarcera população excluídos divisão capital trabalho sendo detentos considerados inimigos estado sendo reservados tratamentos distin

In [77]:
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding = "post",truncating = "post")

In [78]:
print(test_padded)

[[   1    1  139    1 2556  341  199  912 2453    1    1    1   46  120
     1    1    1  327  210  932 2539    1 1964    1   13   64  177    1
     1   48    1 2452 1948    1  341  199   19    2   14  867   13  171
     1  177    1    1 1948  912  341  199    1   68    1    1 1948    1
   769    1 2185 2026   68    1    1    1 2452    1 2539   70  330  538
     1  848   46    1  782    1   67 1951    1  486  124    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  14  867    1 2525    1    1    1    1   24    1   83 2452    1    1
     1    1    1    1  975    2   63    1    1    1  120   63    1    1
     1 2544 1895  111  127    1    1 2549    1    1 1553  867 2544    1
     1   10    1  324 1484  120    1 1385    1  497  335    1 2386    1
     1    1 2458    1  736    1    1   63    1    1    1  232    1  580
     1    1    1    1    1    1  913  670    1  142   24 1770    0    0
     0    0    0    0    0    0    0    0    0    0

In [79]:
predictions_t = model.predict(test_padded)
predictions_t = [1 if p > 0.5 else 0 for p in predictions_t]

In [80]:
print(predictions_t)
print(test_labels)

[0, 0, 0, 0, 1, 0, 0, 1]
[0 0 0 0 1 1 1 1]


In [81]:
decoded_test = decode(test_sequences[7])

In [82]:
print(decoded_test)

<OOV> <OOV> <OOV> elementos <OOV> potência essenciais conexão fontes <OOV> energia <OOV> eólica rede elétrica permitem controle motores trifásicos além serem peça fundamental <OOV> utilização modulação <OOV> <OOV> <OOV> <OOV> importante técnica nesse domínio devido aproveitamento eficiente tensão <OOV> <OOV> geração saída baixa <OOV> <OOV> trabalho visa apresentar forma simples clara métodos implementação modulação <OOV> <OOV> três topologias <OOV> trifásicos dois níveis três <OOV> três <OOV> três <OOV> quatro <OOV> quatro <OOV> quatro <OOV> desenvolvimento algoritmos modulação baseado literatura técnica <OOV> <OOV> <OOV> algoritmos métodos desenvolvidos utilizando software simulação <OOV>


In [83]:
#model.save_weights('m_salvo/')

In [84]:
#model.save('modelo_completo/')